In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import json
from PyForks import Region
import pyarrow
from configparser import ConfigParser

In [ ]:
parser = ConfigParser()
parser.read('../PyForks/_test/secrets.ini')
app_id = parser.get('trailforks', 'app_id')
app_secret = parser.get('trailforks', 'app_secret')
r = Region(app_id=app_id, app_secret=app_secret)

In [ ]:
df = r.get_all_trailforks_regions()

In [ ]:
df.drop_duplicates(subset=['alias'], inplace=True)
df.to_parquet("./PyForks/data/region_data.parquet", engine='pyarrow')

In [ ]:
def get_region_link(row) -> str:
    uri_name = row['region_uri_name']
    link = f"https://www.trailforks.com/region/{uri_name}"
    return link

def get_region_name(row) -> str:
    name = row['name']
    state = row['state']
    country = row['country']
    return f"{name}, {state}, {country}"

In [ ]:
t = df.rename(columns={
    "title":"name", 
    "alias":"region_uri_name", 
    "prov_title":"state", 
    "city_title":"city",
    "country_title":"country"
    })

t['region_link'] = t.apply(get_region_link, axis=1)
t['full_region_name'] = t.apply(get_region_name, axis=1)
t.to_json("trailforks_regions.json", orient='records')

regions = t.full_region_name.to_list()
with open("region_names.json", "w") as f:
    json.dump(regions, f)